In [10]:
import afqinsight.nn.tf_models as nn
import numpy as np
import tensorflow as tf

from afqinsight.datasets import AFQDataset
from afqinsight.nn.tf_models import mlp4, cnn_lenet
from sklearn.impute import SimpleImputer

In [11]:
afq_dataset = AFQDataset(
    fn_nodes="../data/raw/combined_tract_profiles.csv",
    fn_subjects="../data/raw/participants_updated_id.csv",
    dwi_metrics=["dki_fa", "dki_md"],
    target_cols=["age"],
    index_col="subject_id",
)

In [12]:
afq_dataset.drop_target_na()

In [13]:
print(len(afq_dataset.subjects))
print(afq_dataset.X.shape)
print(afq_dataset.y.shape)

1867
(1867, 4800)
(1867,)


In [14]:
# Here we impute missing bundles.
# We impute using the entire dataset, which permits data leakage between the train and test set.
# THIS IS BAD AND SHOULDN'T BE DONE IN PRODUCTION
# But we do it here to move straight to model training
# When we are more comfortabel with the models, we should come back
# here and train the imputer only on the training set.
imputer = SimpleImputer(strategy="median")
afq_dataset.X = imputer.fit_transform(afq_dataset.X)

In [21]:
# Separate into training, test, and validation sets
dataset_size = len(afq_dataset.subjects)
train_size = int(0.7 * dataset_size)
val_size = int(0.15 * dataset_size)
test_size = int(0.15 * dataset_size)

full_dataset = afq_dataset.as_tensorflow_dataset()
full_dataset = full_dataset.shuffle(buffer_size=dataset_size, seed=0)

train_dataset = full_dataset.take(train_size)
test_dataset = full_dataset.skip(train_size)
val_dataset = test_dataset.skip(val_size)
test_dataset = test_dataset.take(test_size)

In [22]:
batch_size = 128
train_dataset = train_dataset.batch(batch_size)
test_dataset = test_dataset.batch(batch_size)
val_dataset = val_dataset.batch(batch_size)

In [23]:
# First let's check out the ``cnn_lenet`` from the tf_models module
model_cnn_lenet= cnn_lenet(input_shape=(100, 48) , n_classes=1, output_activation=None, verbose=True)

pooling layers: 4
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 100, 48)]         0         
                                                                 
 conv1d_4 (Conv1D)           (None, 100, 6)            870       
                                                                 
 max_pooling1d_4 (MaxPooling  (None, 50, 6)            0         
 1D)                                                             
                                                                 
 conv1d_5 (Conv1D)           (None, 50, 16)            304       
                                                                 
 max_pooling1d_5 (MaxPooling  (None, 25, 16)           0         
 1D)                                                             
                                                                 
 conv1d_6 (Conv1D)           (None, 25, 2

In [25]:
# Compile the model and fit it using training data
model_cnn_lenet.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])
model_cnn_lenet.fit(train_dataset, epochs=100, validation_data=val_dataset)

Epoch 1/100
11/11 [==============================] - 1s 40ms/step - loss: 81.5696 - mean_squared_error: 81.5696 - val_loss: 12.5674 - val_mean_squared_error: 12.5674
Epoch 2/100
11/11 [==============================] - 0s 26ms/step - loss: 30.0908 - mean_squared_error: 30.0908 - val_loss: 23.5813 - val_mean_squared_error: 23.5813
Epoch 3/100
11/11 [==============================] - 0s 28ms/step - loss: 26.2477 - mean_squared_error: 26.2477 - val_loss: 11.9040 - val_mean_squared_error: 11.9040
Epoch 4/100
11/11 [==============================] - 0s 26ms/step - loss: 21.9434 - mean_squared_error: 21.9434 - val_loss: 18.9480 - val_mean_squared_error: 18.9480
Epoch 5/100
11/11 [==============================] - 0s 25ms/step - loss: 21.1915 - mean_squared_error: 21.1915 - val_loss: 12.3486 - val_mean_squared_error: 12.3486
Epoch 6/100
11/11 [==============================] - 0s 28ms/step - loss: 20.3403 - mean_squared_error: 20.3403 - val_loss: 13.7637 - val_mean_squared_error: 13.7637
Epoc

In [26]:
# Evaluate the model on the validation set
eval_cnn_lenet = model_cnn_lenet.evaluate(val_dataset)

3/3 [==============================] - 0s 6ms/step - loss: 9.4867 - mean_squared_error: 9.4867


We see that this model achieves a mean squared error of ~6.4, or an RMSE of ~2.5 years. This is probably an overly optimistic estimate because we imputed using the entire dataset.

In [27]:
# define an empty dictionary for results comparisons
results = {'cnn_lenet' : eval_cnn_lenet}

In [28]:
#``mlp4`` from the tf_models module
model_mlp4= mlp4(input_shape=(100, 48) , n_classes=1, output_activation=None, verbose=True)
# Compile the model and fit it using training data
model_mlp4.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])
model_mlp4.fit(train_dataset, epochs=100)
# Evaluate the model on the validation set
eval_mlp4 = model_mlp4.evaluate(val_dataset)

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 100, 48)]         0         
                                                                 
 flatten_3 (Flatten)         (None, 4800)              0         
                                                                 
 dropout_8 (Dropout)         (None, 4800)              0         
                                                                 
 dense_10 (Dense)            (None, 500)               2400500   
                                                                 
 dropout_9 (Dropout)         (None, 500)               0         
                                                                 
 dense_11 (Dense)            (None, 500)               250500    
                                                                 
 dropout_10 (Dropout)        (None, 500)               0   

In [29]:
# update the result
results.update({'mlp4': eval_mlp4})

In [31]:
from afqinsight.nn.tf_models import cnn_vgg, lstm1v0, lstm1, lstm2, blstm1, blstm2, lstm_fcn, cnn_resnet
#``cnn_vgg`` from the tf_models module
model_cnn_vgg= cnn_vgg(input_shape=(100, 48) , n_classes=1, output_activation=None, verbose=True)
# Compile the model and fit it using training data
model_cnn_vgg.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])
model_cnn_vgg.fit(train_dataset, epochs=100)
# Evaluate the model on the validation set
eval_cnn_vgg = model_cnn_vgg.evaluate(val_dataset)
# update the result
results.update({'cnn_vgg': eval_cnn_vgg})

In [17]:
#``lstm1v0`` from the tf_models module
model_lstm1v0= lstm1v0(input_shape=(100, 48) , n_classes=1, output_activation=None, verbose=True)
# Compile themodel and fit it using training data
model_lstm1v0.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])
model_lstm1v0.fit(train_dataset, epochs=100)
# Evaluate the model on the validation set
eval_lstm1v0 = model_lstm1v0.evaluate(val_dataset)
# update the result
results.update({'lstm1v0': eval_lstm1v0})

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 100, 48)]         0         
                                                                 
 lstm (LSTM)                 (None, 512)               1148928   
                                                                 
 dense_13 (Dense)            (None, 1)                 513       
                                                                 
Total params: 1,149,441
Trainable params: 1,149,441
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
11/11 [==============================] - 9s 638ms/step - loss: 36.8216 - mean_squared_error: 36.8216
Epoch 2/100
11/11 [==============================] - 7s 604ms/step - loss: 12.9252 - mean_squared_error: 12.9252
Epoch 3/100
11/11 [==============================] - 7s 672ms/step - loss: 12.4212 - me

In [32]:
#``lstm1`` from the tf_models module
model_lstm1= lstm1(input_shape=(100, 48) , n_classes=1, output_activation=None, verbose=True)
# Compile the model and fit it using training data
model_lstm1.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])
model_lstm1.fit(train_dataset, epochs=100, validation_data=val_batch)
# Evaluate the model on the validation set
eval_lstm1 = model_lstm1.evaluate(val_dataset)
# update the result
results.update({'lstm1': eval_lstm1})

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 100, 48)]         0         
                                                                 
 lstm (LSTM)                 (None, 100)               59600     
                                                                 
 dense_14 (Dense)            (None, 1)                 101       
                                                                 
Total params: 59,701
Trainable params: 59,701
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
11/11 [==============================] - 4s 148ms/step - loss: 102.1830 - mean_squared_error: 102.1830 - val_loss: 56.8872 - val_mean_squared_error: 56.8872
Epoch 2/100
11/11 [==============================] - 1s 120ms/step - loss: 33.6234 - mean_squared_error: 33.6234 - val_loss: 16.8095 - val_mean_square

In [19]:
#``lstm2`` from the tf_models module
model_lstm2= lstm2(input_shape=(100, 48) , n_classes=1, output_activation=None, verbose=True)
# Compile the model and fit it using training data
model_lstm2.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])
model_lstm2.fit(train_dataset, epochs=100)
# Evaluate the model on the validation set
eval_lstm2 = model_lstm2.evaluate(val_dataset)
# update the result
results.update({'lstm2': eval_lstm2})

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 100, 48)]         0         
                                                                 
 lstm_2 (LSTM)               (None, 100, 100)          59600     
                                                                 
 lstm_3 (LSTM)               (None, 100)               80400     
                                                                 
 dense_15 (Dense)            (None, 1)                 101       
                                                                 
Total params: 140,101
Trainable params: 140,101
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
11/11 [==============================] - 6s 205ms/step - loss: 72.6073 - mean_squared_error: 72.6073
Epoch 2/100
11/11 [==============================] - 2s 210ms/step - los

In [20]:
#``blstm1`` from the tf_models module
model_blstm1= blstm1(input_shape=(100, 48) , n_classes=1, output_activation=None, verbose=True)
# Compile the model and fit it using training data
model_blstm1.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])
model_blstm1.fit(train_dataset, epochs=100)
# Evaluate the model on the validation set
eval_blstm1 = model_blstm1.evaluate(val_dataset)
# update the result
results.update({'blstm1': eval_blstm1})

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 100, 48)]         0         
                                                                 
 bidirectional (Bidirectiona  (None, 200)              119200    
 l)                                                              
                                                                 
 dense_16 (Dense)            (None, 1)                 201       
                                                                 
Total params: 119,401
Trainable params: 119,401
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
11/11 [==============================] - 6s 160ms/step - loss: 83.5188 - mean_squared_error: 83.5188
Epoch 2/100
11/11 [==============================] - 2s 146ms/step - loss: 18.1439 - mean_squared_error: 18.1439
Epoch 3/100
11/11 [======

In [21]:
#``blstm2`` from the tf_models module
model_blstm2= blstm2(input_shape=(100, 48) , n_classes=1, output_activation=None, verbose=True)
# Compile the model and fit it using training data
model_blstm2.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])
model_blstm2.fit(train_dataset, epochs=100)
# Evaluate the model on the validation set
eval_blstm2 = model_blstm2.evaluate(val_dataset)
# update the result
results.update({'blstm2': eval_blstm2})

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 100, 48)]         0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 100, 200)         119200    
 nal)                                                            
                                                                 
 bidirectional_2 (Bidirectio  (None, 200)              240800    
 nal)                                                            
                                                                 
 dense_17 (Dense)            (None, 1)                 201       
                                                                 
Total params: 360,201
Trainable params: 360,201
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
11/11 [==============================] - 

In [22]:
#``lstm_fcn`` from the tf_models module
model_lstm_fcn= lstm_fcn(input_shape=(100, 48) , n_classes=1, output_activation=None, verbose=True)
# Compile the model and fit it using training data
model_lstm_fcn.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])
model_lstm_fcn.fit(train_dataset, epochs=100)
# Evaluate the model on the validation set
eval_lstm_fcn = model_lstm_fcn.evaluate(val_dataset)
# update the result
results.update({'lstm_fcn': eval_lstm_fcn})

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, 100, 48)]    0           []                               
                                                                                                  
 conv1d_18 (Conv1D)             (None, 100, 128)     49280       ['input_10[0][0]']               
                                                                                                  
 batch_normalization (BatchNorm  (None, 100, 128)    512         ['conv1d_18[0][0]']              
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 100, 128)     0           ['batch_normalization[0][0]

In [23]:
#``cnn_resnet`` from the tf_models module
model_cnn_resnet= cnn_resnet(input_shape=(100, 48) , n_classes=1, output_activation=None, verbose=True)
# Compile the model and fit it using training data
model_cnn_resnet.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])
model_cnn_resnet.fit(train_dataset, epochs=100)
# Evaluate the model on the validation set
eval_cnn_resnet = model_cnn_resnet.evaluate(val_dataset)
# update the result
results.update({'cnn_resnet': eval_cnn_resnet})

Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, 100, 48)]    0           []                               
                                                                                                  
 conv1d_21 (Conv1D)             (None, 100, 64)      24640       ['input_11[0][0]']               
                                                                                                  
 batch_normalization_3 (BatchNo  (None, 100, 64)     256         ['conv1d_21[0][0]']              
 rmalization)                                                                                     
                                                                                                  
 activation_3 (Activation)      (None, 100, 64)      0           ['batch_normalization_3[0]

In [28]:
results

{'cnn_lenet': [6.719839572906494, 6.719839572906494],
 'mlp4': [16.522377014160156, 16.522377014160156],
 'lstm1v0': [11.4533052444458, 11.4533052444458],
 'lstm1': [12.279282569885254, 12.279282569885254],
 'lstm2': [14.08422565460205, 14.08422565460205],
 'blstm1': [13.48775863647461, 13.48775863647461],
 'blstm2': [13.645526885986328, 13.645526885986328],
 'lstm_fcn': [57.83282470703125, 57.83282470703125],
 'cnn_resnet': [1.352196216583252, 1.352196216583252]}